In [1]:
username = "Alex"
pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")

using StaticArrays, DifferentialEquations, DynamicalSystems, JLD

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`


In [2]:
function model_braint_rhythms(u, p, t)

    sE, sI, rE, rI, Y = u
    τsE, γE, s0E, τsI, γI, s0I, τrE, kE, IE, wEE, wIE, θE, τrI, kI, II, wEI, wII, θI, τY, βY, gammaY, ythr, sEthr, kY = p

    g(Y) = 1 + gammaY / (1 + exp(-Y + ythr))
    HevY(sE) = 1/(1 + exp(-(sE - sEthr)/kY))

    dsEdt = (1/τsE) *(-sE + γE*rE*(1-sE)*g(Y) + s0E)
    dsIdt = (1/τsI) *(-sI + γI*rI*(1-sI) + s0I)

    drEdt = (1/ τrE) *(-rE + 1/(1 + exp(-kE*((IE + wEE*sE-wIE*sI) -  θE))))
    drIdt = (1/τrI) *(-rI + 1/(1 + exp(-kI*((II + wEI*sE-wII*sI) - θI))))
    
    dYdt = -Y / τY + βY * HevY(sE)
    
    return SVector(dsEdt, dsIdt, drEdt, drIdt, dYdt)
end

function output(idx, parameter, u0)
    println("index: $idx; " * name_control_param * ": $parameter");flush(stdout)
    println("u0: $u0");flush(stdout)
end
function output(Λ, u0)
    println("Λ: $Λ");flush(stdout)
    println("last point: $u0");flush(stdout)
end
function separate()
    println("----------------");flush(stdout)
    println("  ");flush(stdout)
end
function save_tofile(name, num_spikes, u0s)
    namenum_spikes = "num_spieks "*  name * ".jld"
    nameu0s = "u0s "*  name * ".jld"
    save(namenum_spikes, "data", num_spikes)
    save(nameu0s, "data", u0s)
end

save_tofile (generic function with 1 method)

In [3]:
function detect_spikes(timeseries)
    obs_var, t_range = timeseries
    local_max, t_local_max = Float64[], Float64[]

    for idx in range(2, length(obs_var) - 1, step = 1)
        if obs_var[idx-1] < obs_var[idx] > obs_var[idx+1]
            push!(local_max, obs_var[idx])
            push!(t_local_max, t_range[idx])
        end
    end
    return local_max, t_local_max
end

detect_spikes (generic function with 1 method)

In [4]:
time = 500.0; tt = 250.0; tstep = 0.001; times = [time, tt]
integ_set = (alg = Vern9(), adaptive = false, dt = tstep);

In [5]:
τsE = 3.0; γE = 4.0; s0E = 0.15;
τsI = 10.0; γI = 8.0; s0I = 0.1;

τrE = 2.0; kE = 5.0; IE = 0.9; wEE = 3.5; wIE = 5.0; θE = 0.2;
τrI = 6.0; kI = 5.0; II = 0.0; wEI = 5.0; wII = 3.0; θI = 0.4;

τY = 0.01;  βY = 0.01; γY = 0.0
ythr = 0.5; sEthr = 0.5; kY = 0.01

params = [τsE, γE, s0E, τsI, γI, s0I, τrE, kE, IE, wEE, wIE, θE, τrI, kI, II, wEI, wII, θI, τY, βY, γY, ythr, sEthr, kY];
u0_start = [0.0, 0.0, 0.0, 0.0, 0.0];

In [ ]:
"""ds = CoupledODEs(model_braint_rhythms,  u0_start, params, diffeq = integ_set)
tr, trange = trajectory(ds, time, Δt = tstep; Ttr = tt)
timeseriesrE = tr[:, 3], trange
timeseriesrI = tr[:, 4], trange
lcmaxrE, tlocalmaxrE = detect_spikes(timeseriesrE)
lcmaxrI, tlocalmaxrI = detect_spikes(timeseriesrI)

f = Figure(resolution = (2000, 400))
axis = Axis(f[1, 1])
lines!(axis, trange, tr[:, 3], linewidth = 0.5, color = :deeppink)
lines!(axis, trange, tr[:, 4], linewidth = 0.5, color = :blue)

scatter!(axis, tlocalmaxrE, lcmaxrE, color = :black)
scatter!(axis, tlocalmaxrI, lcmaxrI, color = :black)
display(f)

println("Number of spieks $(length(lcmaxrE))")
println("Number of spieks $(length(lcmaxrI))")"""

In [6]:
name_control_param = "γy"
index_control_param = 21
start_param = 0.0
end_param = 100.0
length_range = 2001
param_range = range(start_param, end_param, length = length_range)

0.0:0.05:100.0

In [7]:
num_spikes =  zeros(length(param_range), 2)
u0s = zeros(length(param_range), 5)
filename = "2000 change γY to 100.0"

"2000 change γY to 100.0"

In [8]:
for (idx_enum, p) in enumerate(param_range)

    if idx_enum == 1
        global u0 = u0_start
        global params_ = params
    end

    params_[index_control_param] = p
    #output(idx_enum, params_[index_control_param], u0)

    ds = CoupledODEs(model_braint_rhythms,  u0, params_, diffeq = integ_set)

    tr, _ = trajectory(ds, time, Δt = tstep; Ttr = tt)
    u0 = tr[end]; ds = CoupledODEs(model_braint_rhythms,  u0, params_, diffeq = integ_set)

    tr, trange = trajectory(ds, time, Δt = tstep; Ttr = tt)
    timeseriesrE = tr[:, 3], trange
    timeseriesrI = tr[:, 4], trange
    lcmaxrE, tlocalmaxrE = detect_spikes(timeseriesrE)
    lcmaxrI, tlocalmaxrI = detect_spikes(timeseriesrI)

    num_spikes[idx_enum, :] = [length(lcmaxrE), length(lcmaxrI)]
    u0s[idx_enum, :] = u0
    
    #output(Λs[idx_enum, :], u0s[idx_enum, :])
    #separate()
    save_tofile(filename, num_spikes, u0s)
    
end